In [3]:
from main import profile_run
from utillities import check_particle_collisions, execute_collision, run_loop, push_next_collision
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [13]:
%lprun -f profile_run profile_run()

Placing particles
Finding inital collisions
Time elapsed: 25.213149785995483


Timer unit: 1e-06 s

Total time: 25.3 s
File: /home/martin/repos/Exercises_CompPhys/ex1/main.py
Function: profile_run at line 78

Line #      Hits         Time  Per Hit   % Time  Line Contents
    78                                           def profile_run():
    79                                               # https://web.archive.org/web/20140513005858im_/http://www.appneta.com/blog/line-profiler-python/
    80         1          3.0      3.0      0.0      name = "profile_run"
    81         1       3963.0   3963.0      0.0      xi, N, T, R = read_params(para_dir + name)
    82         1        154.0    154.0      0.0      radii = np.ones(N) * R
    83         1         12.0     12.0      0.0      masses = np.ones(N)
    84         1          1.0      1.0      0.0      args = (N, T, radii, masses, xi)
    85         1   25295889.0 25295889.0    100.0      particles, t = run_loop(random_dist, args)

In [2]:
%lprun -f run_loop profile_run()

Placing particles
Finding inital collisions
Time elapsed: 26.3678081035614


Timer unit: 1e-06 s

Total time: 25.6797 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: run_loop at line 270

Line #      Hits         Time  Per Hit   % Time  Line Contents
   270                                           def run_loop(init, args, condition=None, n_check=np.inf, TC=False):
   271         1          8.0      8.0      0.0      tic = time.time()
   272         1    3697117.0 3697117.0     14.4      t, particles, collisions, last_collided = setup_loop(init, args)
   273         1          2.0      2.0      0.0      N, T, radii, masses, xi = args
   274                                           
   275         1          1.0      1.0      0.0      n = 0
   276         1       2611.0   2611.0      0.0      bar = Bar("running simulation", max=T)
   277    119482     103664.0      0.9      0.4      while n < T:
   278    119481    1519303.0     12.7      5.9          col = get_next_col(collisions)
   279    119481     164876.0      1.4      0.6       

In [4]:
%lprun -f execute_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 25.861071586608887


Timer unit: 1e-06 s

Total time: 19.2635 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: execute_collision at line 247

Line #      Hits         Time  Per Hit   % Time  Line Contents
   247                                           def execute_collision(n, t, particles, collisions, last_collided, args, col, TC):
   248     10000      12497.0      1.2      0.1      N, T, radii, masses, xi = args
   249     10000       7925.0      0.8      0.0      t_next, i, j, t_added, col_type  = col
   250     10000     524013.0     52.4      2.7      particles[n+1] = particles[n]
   251     10000      20883.0      2.1      0.1      dt = col[0] - t[n]
   252     10000      12913.0      1.3      0.1      t[n+1] = t_next
   253                                           
   254     10000       6936.0      0.7      0.0      if TC:
   255                                                   xi = tc_check(i, n, t, last_collided, xi)
   256                                             

In [5]:
%lprun -f push_next_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 65.78413438796997


Timer unit: 1e-06 s

Total time: 33.1343 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: push_next_collision at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                           def push_next_collision(particles, n, i, t, collisions, radii):
    32     19448     175008.0      9.0      0.5      wall0 = check_wall_collison(particles[n, i, 0], particles[n, i, 2], radii[i])
    33     19448      79865.0      4.1      0.2      wall1 = check_wall_collison(particles[n, i, 1], particles[n, i, 3], radii[i])
    34     19448      51332.0      2.6      0.2      if wall0 != np.inf: heapq.heappush(collisions, (t+wall0, i, -1, t, "wall0"))
    35     19448      38218.0      2.0      0.1      if wall1 != np.inf: heapq.heappush(collisions, (t+wall1, i, -1, t, "wall1"))
    36     19448    6150890.0    316.3     18.6      dts, cond = check_particle_collisions(particles, n, i, radii)
    37  19448000   13259390.0      0.7  

In [6]:
%lprun -f check_particle_collisions profile_run()

Placing particles
Finding inital collisions
Time elapsed: 31.933687925338745


Timer unit: 1e-06 s

Total time: 7.90496 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: check_particle_collisions at line 134

Line #      Hits         Time  Per Hit   % Time  Line Contents
   134                                           def check_particle_collisions(particles, n, i, radii):
   135     20375      57838.0      2.8      0.7      N = len(radii)
   136     20375     343859.0     16.9      4.3      mask = np.arange(N) != i # remove the particle we are checking against
   137     20375    1831953.0     89.9     23.2      one = np.ones_like(particles[n, mask, 0])
   138     20375     265319.0     13.0      3.4      one_v = np.array([one, one]).T
   139     20375     424829.0     20.9      5.4      R = radii[i] * one + radii[mask]
   140     20375    1079115.0     53.0     13.7      dx = particles[n, mask, :2] - particles[n, i, :2] * one_v
   141     20375     917626.0     45.0     11.6      dv = particles[n, mask, 2:] - particles[n, i, 2:] * one_v
